In [ ]:
from Utils import *

mat = scipy.io.loadmat('Data_sets\Synthesis_MS_train_test\MS.mat')
Train = mat['MS']
mat = scipy.io.loadmat('Data_sets\Synthesis_MS_train_test\Verify.mat')
Verify = mat['Verify']
dt = 0.00025
Fs = 1/dt
f_min=1e-8
f_max = Fs/4
fo = 5
nf = 256

In [ ]:
from Model_def import *


INPUT_SIZE1 = np.shape(Train)[1]
INPUT_SIZE2 = np.shape(Train)[0]
INPUT_SIZE3 = nf

input_img = Input(shape=(INPUT_SIZE2, INPUT_SIZE3, 1))

D1 = int(nf/128)
D2 = int(D1*2)
D3 = int(D1*4)
D4 = int(D1*8)
D5 = int(D1*16)
D6 = int(D1*32)
D7 = int(D1*64)
D8 = int(D1*128)
latent_dim = D4

kernel_size = 5, 5

encoded1 = CBA(input_img, D1, kernel_size, strides = (1, 4), padding = 'same')
# encoded1 = Res_CNN_block(encoded1, filters=D1, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)
#encoded1 = Dropout(0.5)(encoded1)

encoded2 = CBA(encoded1, D2, kernel_size, strides = (1, 4), padding = 'same')
# encoded2 = Res_CNN_block(encoded2, filters=D2, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)
#encoded2 = Dropout(0.5)(encoded2)

encoded3 = CBA(encoded2, D3, kernel_size, strides = (1, 4), padding = 'same')
# encoded3 = Res_CNN_block(encoded3, filters=D3, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)
#encoded3 = Dropout(0.5)(encoded3)

encoded4 = CBA(encoded3, D4, kernel_size, strides = (1, 4), padding = 'same')
# encoded4 = Res_CNN_block(encoded4, filters=D4, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)
#encoded4 = Dropout(0.5)(encoded4)

Xm = Reshape((encoded4.shape[1], D4))(encoded4)

Xm = LSTM(D4, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(Xm)
# Xm = BiLSTM_block(D4, drop_rate=0.1, padding='same', inpR=Xm)
# Xm = Attention_Global()(Xm)
Xm, weightd_D0 = Global_attention(drop_rate=0.1, width=None, name='attentionD0', inpC=Xm) 

# Xm = LSTM(D4, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(Xm)
Xm, weightd_P = SeqSelfAttention(return_attention=True, attention_width= 3, name='attention_P')(Xm)

Xm = Reshape((encoded4.shape[1], encoded4.shape[2], D4))(Xm)
# encoded5 = channel_attention(encoded4, ratio=8)

# Decoder
decoded1 = CTBA(Xm, D4, kernel_size, strides = (1, 4), padding = 'same')
# decoded1 = Res_CNN_block(decoded1, filters=D4, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)

decoded2 = CTBA(decoded1, D3, kernel_size, strides = (1, 4), padding = 'same')
# decoded2 = Res_CNN_block(decoded2, filters=D3, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)

decoded3 = CTBA(decoded2, D2, kernel_size, strides = (1, 4), padding = 'same')
# decoded3 = Res_CNN_block(decoded3, filters=D2, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)

decoded4 = CTBA(decoded3, D1, kernel_size, strides = (1, 4), padding = 'same')
# decoded4 = Res_CNN_block(decoded4, filters=D1, kernel_size=3, padding='same', activation='relu', drop_rate=0.1)

decoded = Conv2D(1, kernel_size, strides = (1,1), padding = 'same', activation='linear', name = 'visualized_layer')(decoded4)

encoder = Model(input_img, Xm, name='encoder')
autoencoder = Model(input_img, decoded)


sgd = optimizers.Adam(learning_rate=0.001, decay=0.0, beta_1=0.9, beta_2=0.999, epsilon=1e-08, amsgrad=False)
autoencoder.compile(optimizer=sgd, loss='mse')

autoencoder.summary()

In [ ]:
batch = 1

Train1 = np.transpose(Train)
scal = np.zeros((np.shape(Train1)[0], np.shape(Train1)[1], nf))
for ix in range(0, np.shape(Train1)[0]):
    sig0 = Train1[ix, :]
    temp0 = cwt(sig0, dt, 4, f_min, f_max, nf=nf,  wl='morlet')
    temp0 = np.abs(temp0)
    scal[ix] = np.transpose(temp0)

Verify_1 = np.transpose(Verify)
Verify_cwt = np.zeros((np.shape(Verify_1)[0], np.shape(Verify_1)[1], nf))
for ix in range(0, np.shape(Verify_1)[0]):
    sig0 = Verify_1[ix, :]
    temp0 = cwt(sig0, dt, 4, f_min, f_max, nf=nf,  wl='morlet')
    temp0 = np.abs(temp0)
    Verify_cwt[ix] = np.transpose(temp0)
scal1 = np.reshape(scal, (np.shape(scal)[0], np.shape(scal)[1], np.shape(scal)[2], 1))
Verify_cwt1 = np.reshape(Verify_cwt, (np.shape(Verify_cwt)[0], np.shape(Verify_cwt)[1], np.shape(Verify_cwt)[2], 1))

autoencoder.fit(scal1, scal1, epochs=5, batch_size=batch, shuffle=False, verbose=1)  # validation_data=(Verify_cwt1, Verify_cwt1),

In [ ]:
mat = scipy.io.loadmat('Data_sets\Synthesis_MS_train_test\test.mat')
Test = mat['test']

Test1 = np.transpose(Test)
Test_cwt = np.zeros((np.shape(Test1)[0], np.shape(Test1)[1], nf))
for ix in range(0, np.shape(Test1)[0]):
    sig0 = Test1[ix, :]
    temp0 = cwt(sig0, dt, 4, f_min, f_max, nf=nf,  wl='morlet')
    temp0 = np.abs(temp0)
    Test_cwt[ix] = np.transpose(temp0)


min_max_scaler = preprocessing.MaxAbsScaler()
Test1 = min_max_scaler.fit_transform(Test1)
clabelf, labelproposed = FCM_predict(Test1, encoder, Test_cwt, nf, D4)